# Melhora dos Dados Estatísticos

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

### Dataframe

In [ ]:
df_pyspark = spark.read.csv('dados2.csv', header=True, inferSchema=True)
df_pyspark.show()

In [ ]:
# Descrever os valores estatísticos
df_pyspark.describe().show()

In [ ]:
# Executar uma única vez
!pip install pandas

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def describe_pd(df_in, columns, deciles=False):
    if deciles:
        percentiles = np.array(range(0, 110, 10))
    else:
        percentiles = [25, 50, 75]
    percs = np.transpose([np.percentile(df_in.select(x).collect(),percentiles) for x in columns])
    percs = pd.DataFrame(percs, columns=columns)
    percs['summary'] = [str(p) + '%' for p in percentiles]
    spark_describe = df_in.describe().toPandas()
    new_df = pd.concat([spark_describe, percs],ignore_index=True)
    new_df = new_df.round(2)
    return new_df[['summary'] + columns]

In [ ]:
describe_pd(df_pyspark, ['Idade', 'Experiencia'])

### Assimetria e Curtose

In [ ]:
from pyspark.sql.functions import col, skewness, kurtosis

In [ ]:
df_pyspark.select(skewness(df_pyspark.Idade),kurtosis(df_pyspark.Idade)).show()

In [ ]:
# Executar uma única vez
!pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
coluna_selecionada = df_pyspark.select("Idade")
x = coluna_selecionada.rdd.flatMap(lambda x: x).collect()

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(x, kde=True)
plt.xlabel('Idade')
plt.ylabel('Quantidade')
plt.show()

In [ ]:
# Finalizar a Seção
spark.sparkContext.stop()